In [1]:
%load_ext sql

In [2]:
%sql sqlite:///./test.db

'Connected: @./test.db'

In [3]:
%%sql
drop table if exists test_match;
create table test_match(team_1 string,team_2 string, result string);

 * sqlite:///./test.db
Done.
Done.


[]

In [5]:
%%sql
insert into test_match values
('India','Australia','India'),
('India','England','England'),
('SouthAfrica','India','India'),
('Australia','England',Null),
('England','SouthAfrica','SouthAfrica'),
('Australia','India','Australia')
;

 * sqlite:///./test.db
6 rows affected.


[]

In [6]:
%sql select * from test_match;

 * sqlite:///./test.db
Done.


team_1,team_2,result
India,Australia,India
India,England,England
SouthAfrica,India,India
Australia,England,None
England,SouthAfrica,SouthAfrica
Australia,India,Australia


SQL to Display total number of matches played,matches won,matches won and matches lost for each of them

In [28]:
%%sql

select base.team,base.total_matches,win.total_wins,loss.total_losses,ifnull(tie.tied,0) tied from
(select team,count(*) total_matches from (
select team_1 team from test_match
union all
select team_2 team from test_match
) b group by team) base
join 
(
select result as winner,count(*) as total_wins from test_match w group by winner
) win 
on base.team=win.winner
join
(
select case when result=team_1 then team_2
     when result=team_2 then team_1
else null end loser,count(*) as total_losses from test_match l group by 
case when result=team_1 then team_2
     when result=team_2 then team_1
else null end    
) loss
on base.team=loss.loser
left join 
(
select team,count(*) tied from (    
select team_1 team from test_match where result is null
union all
select team_2 team from test_match where result is null
)t group by team )tie
on base.team=tie.team

 * sqlite:///./test.db
Done.


team,total_matches,total_wins,total_losses,tied
Australia,3,1,1,1
England,3,1,1,1
India,4,2,2,0
SouthAfrica,2,1,1,0
